# 06_merge.ipynb

In [25]:
import pandas as pd
import geopandas as gpd
from dask.distributed import Client
import dask.dataframe as dd
from dask_jobqueue import SLURMCluster
import matplotlib.pyplot as plt
import contextily as cx
from build_utilities import generate_variable_names
import dask_geopandas
INPUT_DATA_EVICTIONS = "../../data/02_intermediate/evictions.csv"
INPUT_DATA_TRACTS = "../../data/02_intermediate/tracts.csv"
INPUT_DATA_TAX_PARCELS = "../../data/02_intermediate/tax_parcels.gpkg"
INPUT_DATA_ZESTIMATES = "../../data/02_intermediate/zestimates.csv"
INPUT_DATA_CRIME = "../../data/01_raw/crime_incidents"
OUTPUT_DATA_UNRESTRICTED = "../../data/03_cleaned/unrestricted.csv"
OUTPUT_DATA_ZILLOW = "../../data/03_cleaned/zestimates_analysis.csv"
OUTPUT_DATA_CRIME = "../../data/03_cleaned/crime_analysis.csv"
VERBOSE = True
N_PARTITIONS = 1

## 1. Loading Evictions Data

In [26]:
# Load evictions data.
with open(INPUT_DATA_EVICTIONS, 'r') as file:
    all_column_names = set(file.readline().replace("\"", "").replace("\n", "").split(","))
to_drop = {'Accuracy Score', 'Accuracy Type', 'Number', 'Street', 'Unit Type', 'Unit Number',
           'State', 'Zip', 'Country', 'Source', 'Census Year', 'State FIPS', 'County FIPS',
           'Place Name', 'Place FIPS', 'Census Tract Code', 'Census Block Code', 'Census Block Group',
           'Metro/Micro Statistical Area Code', 'Metro/Micro Statistical Area Type',
           'Combined Statistical Area Code', 'Metropolitan Division Area Code', 'court_location',
           'defendant', 'defendant_atty', 'defendant_atty_address_apt',
           'defendant_atty_address_city', 'defendant_atty_address_name', 'defendant_atty_address_state',
           'defendant_atty_address_street', 'defendant_atty_address_zip', 'docket_history', 'execution', 'judgment_for',
           'judgment_total', 'latest_docket_date', 'plaintiff', 'plaintiff_atty', 'plaintiff_atty_address_apt',
           'plaintiff_atty_address_city', 'plaintiff_atty_address_name', 'plaintiff_atty_address_state',
           'plaintiff_atty_address_street', 'plaintiff_atty_address_zip', 'Metropolitan Division Area Name',
           'property_address_city', 'property_address_state', 'property_address_street',
           'property_address_zip'}
df = pd.read_csv(INPUT_DATA_EVICTIONS, usecols=set(all_column_names) - set(to_drop))
original_N = len(df)
if VERBOSE:
    print(f"Beginning with {original_N} observations.")

# Drop cases missing file_date.
mask = df['file_date'].notna()
if VERBOSE:
    print(
        f"Dropping {(~mask).sum()} observations where file_date is missing.")
df = df.loc[mask, :]

# Add file month and year to dataset.
df.loc[:, 'file_month'] = pd.to_datetime(df['file_date']).dt.strftime('%Y-%m')
df.loc[:, 'file_year'] = pd.to_datetime(df['file_date']).dt.year

# Clean the values in the judgment_for_pdu variable.
judgment_for_pdu_replacement_dict = {"unknown": "Unknown",
                                     "plaintiff": "Plaintiff",
                                     "defendant": "Defendant"}
df.loc[:, "judgment_for_pdu"] = (df.loc[:, "judgment_for_pdu"]
                                           .replace(judgment_for_pdu_replacement_dict))

# Replace missing values in money judgment column with zeroes.
df.loc[:, 'judgment'] = df['judgment'].fillna(0)

# Rename duration to case_duration.
df = df.rename(columns={'duration': 'case_duration'})

# Drop malformed addresses.
if VERBOSE:
    print(f"Dropping {df['property_address_full'].str.contains('span, span span').sum()} observations which "
          f"have malformed addresses.")
df = df.loc[~df['property_address_full'].str.contains("span, span span"), :]

# Drop addresses without latitude and longitude.
if VERBOSE:
    print(f"Dropping {df[['longitude', 'latitude']].isna().any(axis=1).sum()} evictions missing latitude "
          f"or longitude.")
df = df.dropna(subset=['longitude', 'latitude'])

Beginning with 40759 observations.
Dropping 0 observations where file_date is missing.
Dropping 24 observations which have malformed addresses.
Dropping 1 evictions missing latitude or longitude.


## 2. Merging Evictions With Census Tract Characteristics

In [27]:
# Merge with census tract characteristics.
df = df.rename(columns={'Full FIPS (tract)': 'tract_geoid'})
df = df.merge(pd.read_csv(INPUT_DATA_TRACTS, dtype={'tract_geoid': float}),
                                  on='tract_geoid',
                                  how='left',
                                  validate='m:1')
if VERBOSE:
    print(f"Successfully merged {df['med_hhinc2016'].notna().sum()} observations "
          f"({df['med_hhinc2016'].notna().sum() / original_N:.2f} percent of observations) with census "
          f"tracts.")

Successfully merged 40732 observations (1.00 percent of observations) with census tracts.


## 3. Merging Evictions With Zestimates

In [28]:
df = pd.read_csv(INPUT_DATA_ZESTIMATES).merge(df,
                                                     on='case_number',
                                                     how='right',
                                                     validate='1:1')
if VERBOSE:
    successfully_matched_observations = (~df['2022-12'].isna()).sum()
    print(
        f"Successfully matched {successfully_matched_observations} evictions "
        f"({100 * (successfully_matched_observations / len(df)) :.2f} percent of observations) to "
        f"Zestimates.")

# Rename columns containing Zestimates.
years = [str(year) for year in range(2013, 2023)]
months = ["0" + str(month) for month in range(1, 10)] + [str(month) for month in range(10, 13)]
value_vars = ["2012-12"] + [str(year) + "-" + str(month) for year in years for month in months]
value_vars_zestimates, _, _ = generate_variable_names('zestimate')
for value_var, value_var_zestimates in zip(value_vars, value_vars_zestimates):
    df = df.rename(columns={value_var: value_var_zestimates})

Successfully matched 11496 evictions (28.22 percent of observations) to Zestimates.


## 4. Merging Evictions with Tax Parcels

In [5]:
# Request computing resources.
cluster = SLURMCluster(queue='batch',
                       cores=32,
                       memory='230 GB',
                       walltime='00:30:00',
                      scheduler_options={'dashboard_address': '8787'} )
cluster.scale(jobs=1)


In [6]:
client = Client(cluster)

In [ ]:
df = gpd.GeoDataFrame(df,
                          geometry=gpd.points_from_xy(df['longitude'], df['latitude']))

df = df.set_crs("EPSG:4326", allow_override=True).to_crs('EPSG:26986')
ddf = dask_geopandas.from_geopandas(df, npartitions=N_PARTITIONS).repartition(partition_size='50 MB')

tax_parcels_dgdf = dask_geopandas.read_file(INPUT_DATA_TAX_PARCELS, npartitions=N_PARTITIONS, layer='layer').repartition(partition_size='50 MB')

ddf = dask_geopandas.sjoin(tax_parcels_dgdf, ddf, how='inner', predicate='contains')
ddf = ddf.loc[ddf['LOC_ID'] != "F_819960_2934955", :]  # Drop the eviction which erroneously merges to two parcels. 
ddf = ddf.drop(columns=['index_right']).compute()

# Append the evictions which could not be merged to tax parcels back into the Dask DataFrame.
evictions_without_parcels = df.loc[~(df['case_number'].isin(ddf['case_number'])), :]
ddf = pd.concat([ddf, evictions_without_parcels], axis=0)

## 5a. Merge Evictions With Own-Parcel Crime Data

In [ ]:
# Convert eviction data to Dask-GeoDataFrame.
ddf = dask_geopandas.from_geopandas(ddf, npartitions=N_PARTITIONS)
ddf = ddf.repartition(partition_size='25 MB')  # Reduce partition size in preparation for spatial join.

# Read crime data as Dask DataFrame; clean it.
crime_dgdf = (dd.read_csv(INPUT_DATA_CRIME + "/*.csv", dtype={'REPORTING_AREA': 'object', 'SHOOTING': 'object'})
                .dropna(subset=['Long', 'Lat', 'OCCURRED_ON_DATE'])
                .rename(columns={'OCCURRED_ON_DATE': 'month_of_crime_incident'})
                .drop(columns=['OFFENSE_CODE', 'OFFENSE_CODE_GROUP', 'OFFENSE_DESCRIPTION', 'DISTRICT', 'REPORTING_AREA', 'SHOOTING', 'YEAR', 'MONTH',
                               'DAY_OF_WEEK', 'HOUR', 'UCR_PART', 'STREET', 'Location']))
crime_dgdf['month_of_crime_incident'] = dd.to_datetime(crime_dgdf['month_of_crime_incident'].str[:10]).dt.to_period("M").astype(str)
crime_dgdf = crime_dgdf.compute()

# Convert crime data to GeoDataFrame.
crime_dgdf = (gpd.GeoDataFrame(crime_dgdf, geometry=gpd.points_from_xy(crime_dgdf['Long'], crime_dgdf['Lat']))
                               .set_crs("EPSG:4326", allow_override=True)
                               .to_crs("EPSG:26986"))

# Convert crime data to Dask-GeoDataFrame.
crime_dgdf = dask_geopandas.from_geopandas(crime_dgdf, npartitions=N_PARTITIONS).repartition(partition_size='25 MB')
crime_dgdf = crime_dgdf.dissolve(by='INCIDENT_NUMBER').reset_index()

# Join Dask-GeoDataFrames containing eviction data and crime data.
ddf = dask_geopandas.sjoin(crime_dgdf,
                            ddf,
                            how='inner',
                            predicate='within').compute()

# We no longer have any use for crime incident geometry.
ddf = ddf.drop(columns='geometry')

## 5b. Aggregating Own-Parcel Crime Data to Case-Month Level 

In [ ]:
ddf = aggregate_crime_to_case_month(ddf)

In [ ]:
# Now, ddf contains wide format crime incident data for the 792 properties which experienced crime.
# Merge with the evictions that experienced no crime, stored in df.
ddf = ddf.set_index('case_number')
df = df.set_index('case_number')
ddf = pd.concat([df, ddf], axis=1)

In [12]:
# Rename columns containing own-parcel crime incident counts.
years = [str(year) for year in range(2015, 2023)]
months = ["0" + str(month) for month in range(1, 10)] + [str(month) for month in range(10, 13)]
value_vars = [str(year) + "-" + str(month) for year in years for month in months]
value_vars = value_vars[5:]
value_vars.append('2023-01')
value_vars_crimes_own_parcel, _, _ = generate_variable_names('crimes_own_parcel')
for value_var, value_var_crimes_own_parcel in zip(value_vars, value_vars_crimes_own_parcel):
    ddf = ddf.rename(columns={value_var: value_var_crimes_own_parcel})

In [13]:
# Replace missing crime data with zero for evictions that were not matched to crimes.
ddf.loc[:, value_vars_crimes_own_parcel] = ddf[value_vars_crimes_own_parcel].fillna(0)

In [14]:
# Separately store own-parcel crime counts.
to_concat = []
crimes_own_parcel_data = ddf[value_vars_crimes_own_parcel]
to_concat.append(crimes_own_parcel_data)
ddf = ddf.drop(columns=value_vars_crimes_own_parcel)  # Drop from ddf so that we are not spatially joining unnecessary data.
ddf = ddf[['longitude', 'latitude']]  # Can merge eviction data later; only need geometry column for spatial join.

## 6a. Merge Evictions with Crimes Within Varying Distances

`
ddf : GeoDataFrame with only a geometry column, containing Points corresponding to evictions.


for each radius in [60, 90, 140, 200]:
    60m_gdf = ddf.copy()
    60m_gdf.geometry = 60m_gdf.geomtry.buffer(radius)
    
`

In [ ]:
# TODO: perform merge at each value of radius

# Drop existing geometry column and create a new one using the latitude and longitude of the eviction coordinates.
ddf = ddf.drop(columns='geometry')
ddf = gpd.GeoDataFrame(ddf,
                          geometry=gpd.points_from_xy(ddf['longitude'], ddf['latitude']))

ddf = ddf.set_crs("EPSG:4326", allow_override=True).to_crs('EPSG:26986')




ddf.geometry = ddf.geometry.buffer(500)  # Add 500m buffer around every eviction. 
ddf = dask_geopandas.from_geopandas(ddf, npartitions=N_PARTITIONS).repartition(partition_size='50 MB')
# ddf is now a dask-geodataframe where each eviction's geometry is a 500



## 7. Producing the Unrestricted Dataset

In [ ]:
unrestricted_df = pd.DataFrame(unrestricted_df.drop(columns='geometry'))
unrestricted_df.to_csv(OUTPUT_DATA_UNRESTRICTED)

## 8. Producing the Samples Used in Analysis

In [ ]:
# Drop cases resolved via mediation.
mediated_mask = unrestricted_df['disposition_found'] == "Mediated"
if VERBOSE:
    print(f"Dropping {mediated_mask.sum()} cases resolved through mediation.")
unrestricted_df = unrestricted_df.loc[~mediated_mask, :]

In [ ]:
# Drop cases resolved via volntary dismissal (dropped by plaintiff). 
voluntary_dismissal_mask = unrestricted_df['disposition'].str.contains("R 41(a)(1) Voluntary Dismissal on", na=False, regex=False)
if VERBOSE:
    print(f"Droppping {voluntary_dismissal_mask.sum()} cases resolved through voluntary dismissal.")
unrestricted_df = unrestricted_df.loc[~voluntary_dismissal_mask, :]

In [ ]:
# Drop cases where disposition found is other.
disposition_found_other_mask = unrestricted_df['disposition_found'] == "Other"
if VERBOSE:
    print(f"Dropping {disposition_found_other_mask.sum()} cases where disposition_found is \"Other\"")
unrestricted_df = unrestricted_df.loc[~disposition_found_other_mask, :]

In [ ]:
# Drop rows which contain consistent values of disposition_found and judgment_for_pdu.

# First, we drop cases where disposition_found is "Defaulted" but judgment_for_pdu is "Defendant"
inconsistent_mask_1 = ((unrestricted_df['disposition_found'] == "Defaulted") & (unrestricted_df['judgment_for_pdu'] == "Defendant"))
if VERBOSE:
    print(f"Dropping {inconsistent_mask_1.sum()} observations where disposition_found is \"Defaulted\" but judgment_for_pdu is \"Defendant\".")
unrestricted_df = unrestricted_df.loc[~inconsistent_mask_1, :]
          
# Next, we drop cases where disposition_found is "Dismissed" yet judgment_for_pdu is "Plaintiff"
inconsistent_mask_2 = ((unrestricted_df['disposition_found'] == "Dismissed") & (unrestricted_df['judgment_for_pdu'] == "Plaintiff"))
if VERBOSE:
    print(f"Dropping {inconsistent_mask_2.sum()} observations where disposition_found is \"Dismissed\" but judgment_for_pdu is \"Plaintiff\".")
unrestricted_df = unrestricted_df.loc[~inconsistent_mask_2, :]

In [ ]:
# Generate a variable indicating judgment in favor of defendant.
unrestricted_df.loc[:, 'judgment_for_defendant'] = 0
defendant_won_mask = ((unrestricted_df['disposition_found'] == "Dismissed") |
                      (unrestricted_df['judgment_for_pdu'] == "Defendant"))
unrestricted_df.loc[defendant_won_mask, 'judgment_for_defendant'] = 1

# Generate a variable indicating judgement in favor of plaintiff.
unrestricted_df.loc[:, 'judgment_for_plaintiff'] = 1 - unrestricted_df['judgment_for_defendant']


### 8a. Producing the Zillow Sample

In [ ]:
# Drop rows where we are missing any Zestimates.
zestimates_df = unrestricted_df.copy()
has_all_zestimates_mask = zestimates_df[value_vars_zestimates].notna().all(axis=1)
if VERBOSE:
    print(f"Limiting sample to {has_all_zestimates_mask.sum()} evictions for which we observe Zestimates at every month from 2012-12 to 2022-12.")
zestimates_df = zestimates_df.loc[has_all_zestimates_mask, :]
zestimates_df.to_csv(OUTPUT_DATA_ZILLOW)

### 8b. Producing the Crime Sample

In [ ]:
crime_df = unrestricted_df.copy()
# Restrict to evictions which took place in Boston.
boston_mask = ((crime_df['County'] == "Suffolk County") & (~crime_df['City'].isin(["Chelsea", "Revere", "Winthrop"])))
if VERBOSE:
    print(f"Limiting sample to {boston_mask.sum()} observations which are in Boston.")
crime_df = crime_df.loc[boston_mask, :]
crime_df.to_csv(OUTPUT_DATA_CRIME)